In [1]:
# | default_exp solvemodels

## Setup

In [2]:
# | export


import json
import sys

import cobra

# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.buildingediting
import mmon_gcm.solving
import mmon_gcm.supermodel
import pandas as pd

In [3]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is where to adjust whether fva is run if running in notebook
args = {
    "run_fva": "True",
    "no_processes": 3,
    "output_dir": "../outputs/model_solutions/",
    "model_file": "../models/4_stage_GC.xml",
    "map_file": "../inputs/map.json",
    "parameters_file": "../inputs/arabidopsis_parameters.csv",
}

sys.argv = ["script_name"] + list(args.values())

In [4]:
# | export

run_fva = sys.argv[1]
if run_fva == "True":
    run_fva = True
elif run_fva == "False":
    run_fva = False
else:
    raise ValueError(f"Please specify True or False for run_fva, not {run_fva}")

no_processes = int(sys.argv[2])
output_dir = sys.argv[3]
model_file = sys.argv[4]
map_file = sys.argv[5]
parameters_file = sys.argv[6]

## Instantiate and apply base constraints to model

### Import FBA model

In [5]:
# | export

four_stage_GC_model = cobra.io.sbml.read_sbml_model(model_file)  # read model

No objective coefficients in model. Unclear what should be optimized


Decrease tolerance

In [6]:
# | export

print(four_stage_GC_model.solver.configuration.tolerances.integrality)
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)
four_stage_GC_model.solver.configuration.tolerances.feasibility = (
    1e-8  # 1e-9 takes a long time to solve
)
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)

1e-07
1e-07
1e-08


### Define reactions to use for FVA

In [7]:
# | export

if run_fva == True:
    linker_list = [
        reaction
        for reaction in four_stage_GC_model.reactions
        if "gc_Linker_2" in reaction.id
        or "gc_Linker_1" in reaction.id
        or "_total_pseudolinker_1" in reaction.id
        or "_total_pseudolinker_2" in reaction.id
        or "ae_gc" in reaction.id
    ]

    extra_fva_rxns = set()

    with open(map_file, "r+") as f:
        map_data = json.load(f)

    for reaction in map_data[1]["reactions"].values():
        extra_fva_rxns.add(four_stage_GC_model.reactions.get_by_id(reaction["bigg_id"]))

    fva_list = list(set(linker_list) | extra_fva_rxns)

else:
    fva_list = []

### Instantiate Supermodel

Import parameters from csv, created in [parameters notebook](02_parameters.ipynb)

In [8]:
# | export

parameters_df = pd.read_csv(parameters_file, index_col=0)
parameters_df

,Value,Units,Source
P_abs,9.000000e-01,Dimensionless,"Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,m,Wuyts et al. (2010)
A_l,1.000000e+00,m$^2$,Fixed
V_gc_ind,4.750000e-13,dm$^3$,Jezek and Blatt (2017)
FqFm,9.000000e-01,Dimensionless,Lawson (2003)
R_ch,6.923077e-02,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)"
L_air,3.700000e-01,Dimensionless,Earles et al. (2018)
L_epidermis,1.500000e-01,Dimensionless,Wuyts et al. (2010)
Vac_frac,7.510000e-01,Dimensionless,Wang et al. (2017)
T,2.961500e+02,K,Horrer et al. (2016)


In [9]:
# | export

arabidopsis_supermodel = mmon_gcm.supermodel.SuperModel(
    parameters_df.loc[:, "Value"], fba_model=four_stage_GC_model
)

In [10]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True);

Volume in phase 0 is 0.38pL
Volume in phase 1 is 0.4375pL
Volume in phase 2 is 0.38pL
Volume in phase 3 is 0.38pL


This is a little different to Jezek and Blatt (2017), where they have closed as 0.3-0.4pL and open as 0.5-0.65. Open apertures are on the small side but still within their range (2-6). Wang used a volume of 0.4pL.

### Constrain SuperModel

#### Constrain osmolarity of the model using the equation from the OnGuard model (Hills et al 2012)

In [11]:
# | export

arabidopsis_supermodel.constrain_osmolarity(printouts=True);

Raw osmolarities: [0.039359327616195004, 0.05492236409486689, 0.039359327616195004, 0.039359327616195004]
Osmolarities in mM: [178.58134127 216.44281417 178.58134127 178.58134127]
Change in osmolarity: 37.86147289566091mM
c osmolarities constrained to [0.00980047 0.01367567 0.00980047 0.00980047]
v osmolarities constrained to [0.02955886 0.0412467  0.02955886 0.02955886]


In [12]:
import numpy as np

In [13]:
np.array(arabidopsis_supermodel.get_osmolarities())

array([0.03935933, 0.05492236, 0.03935933, 0.03935933])

In [14]:
help(arabidopsis_supermodel.get_osmolarities)

Help on method get_osmolarities in module mmon_gcm.supermodel:

get_osmolarities(apertures='default', equation='onguard', printouts=False) method of mmon_gcm.supermodel.SuperModel instance
    Calculates the osmolarities at each phase of the model using the equation
    specified, default is onguard. Adds these as an attribute to the SuperModel



In [15]:
mmoles_per_m2 = np.array(arabidopsis_supermodel.get_osmolarities())
mmoles_per_gc = mmoles_per_m2 / arabidopsis_supermodel.N_gcs
moles_per_gc = mmoles_per_gc * 10**-3
fmoles_per_gc = moles_per_gc / 10**-15
print(f"Closed fmoles: {fmoles_per_gc[0]}")
print(f"Open fmoles: {fmoles_per_gc[1]}")
print(f"Increase in fmoles: {fmoles_per_gc[1] - fmoles_per_gc[0]}")

Closed fmoles: 67.86090968309483
Open fmoles: 94.69373119804636
Increase in fmoles: 26.832821514951533


#### Constrain photons using a PPFD of 150µmolm$^{-2}$s$^{-1}$, same as used in Horrer et al (2016)

In [16]:
# | export

arabidopsis_supermodel.constrain_photons(150, printouts=True);

Total leaf volume: 0.17dm3
Guard cell volume in 1m2 leaf: 0.0003dm3
Mesophyll cell volume in 1m2 leaf: 0.091dm3
Proportion of the leaf that is gc: 0.003
PPFD 150umolphotonsm-2
Photon influx into leaf: 486.0 mmolphotonsm-2hr-1
e = 0.062
Photon influx into Guard cells: 0.092mmolphotonsm-2hr-1
Photon influx into mesophyll cells: 485.908mmolphotonsm-2hr-1


In [17]:
# | export

arabidopsis_supermodel.add_maintenance();

## Blue Light

### ATPase unconstrained

#### WT

In [18]:
# | export

with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0

    (
        blue_unconstrained_wt,
        blue_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 10:45:00.181202
Running pFBA
Running FVA on 348 reactions using 3 processes
MAL_ae_gc_2
K_c_gc_Linker_1
METHYLENE_THF_pc_gc_2
MET_v_gc_Linker_2
GLU6PDEHYDROG_RXN_p_gc_2
MALATE_DEH_RXN_c_gc_2
MAL_total_pseudolinker_1
UDPKIN_RXN_c_gc_2
1TRANSKETO_RXN_p_gc_2
K_c_gc_Linker_2
FRU_vc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
SUCROSE_c_gc_Linker_1
MAL_total_pseudolinker_2
ASPAMINOTRANS_RXN_m_gc_2
SER_pc_gc_2
MALIC_NADP_RXN_p_gc_2
MALATE_DEH_RXN_p_gc_2
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_1
SUCROSE_c_gc_Linker_2
PHOSPHORIBULOKINASE_RXN_p_gc_2
OAA_MAL_mc_gc_2
ASPAMINOTRANS_RXN_c_gc_2
RXN_1781_v_gc_2
GLC_ae_gc_2
Cl_ae_gc_2
RXN_2141_p_gc_2
SER_v_gc_Linker_2
GDPKIN_RXN_c_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
LEU_v_gc_Linker_2
RXN490_3650_p_gc_2
CITSYN_RXN_m_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
LYS_v_gc_Linker_1
CIT_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
LYS_v_gc_Linker_2
MAL_c_gc_Linker_2
FRU_total_pseudolinker_1
MAL_c_gc_Linker_1
Cl_ae_gc_1
GLC_ae

In [19]:
# | export

blue_unconstrained_wt_solution.to_csv(output_dir + f"blue_unconstrained_wt.csv")

#### Starch KO

In [20]:
# | export

with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        blue_unconstrained_starchko,
        blue_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 11:01:29.496786
Running pFBA
Running FVA on 348 reactions using 3 processes
MAL_ae_gc_2
K_c_gc_Linker_1
METHYLENE_THF_pc_gc_2
MET_v_gc_Linker_2
GLU6PDEHYDROG_RXN_p_gc_2
MALATE_DEH_RXN_c_gc_2
MAL_total_pseudolinker_1
UDPKIN_RXN_c_gc_2
1TRANSKETO_RXN_p_gc_2
K_c_gc_Linker_2
FRU_vc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
SUCROSE_c_gc_Linker_1
MAL_total_pseudolinker_2
ASPAMINOTRANS_RXN_m_gc_2
SER_pc_gc_2
MALIC_NADP_RXN_p_gc_2
MALATE_DEH_RXN_p_gc_2
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_1
SUCROSE_c_gc_Linker_2
PHOSPHORIBULOKINASE_RXN_p_gc_2
OAA_MAL_mc_gc_2
ASPAMINOTRANS_RXN_c_gc_2
RXN_1781_v_gc_2
GLC_ae_gc_2
Cl_ae_gc_2
RXN_2141_p_gc_2
SER_v_gc_Linker_2
GDPKIN_RXN_c_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
LEU_v_gc_Linker_2
RXN490_3650_p_gc_2
CITSYN_RXN_m_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
LYS_v_gc_Linker_1
CIT_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
LYS_v_gc_Linker_2
MAL_c_gc_Linker_2
FRU_total_pseudolinker_1
MAL_c_gc_Linker_1
Cl_ae_gc_1
GLC_ae

In [21]:
# | export

blue_unconstrained_starchko_solution.to_csv(
    output_dir + "blue_unconstrained_starchko.csv"
)

### Constrained ATPase

#### WT

In [22]:
# | export

with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )

    (
        blue_constrained_wt,
        blue_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 11:18:25.458251
Running pFBA
Running FVA on 348 reactions using 3 processes
L_ALPHA_ALANINE_v_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
RXN_10773_c_gc_2
L_ALPHA_ALANINE_v_gc_Linker_2
GLC_PROTON_ec_gc_2
MALIC_NADP_RXN_c_gc_2
MAL_SO4_mc_gc_2
MALATE_DEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
DHAP_3PGA_pc_gc_2
GLY_v_gc_Linker_1
H2O_pc_gc_2
GLY_v_gc_Linker_2
GLUC1PURIDYLTRANS_RXN_c_gc_2
H_pc_gc_2
H_mc_gc_2
Cl_c_gc_Linker_1
NADPHox_p_tx_gc_2
2PGADEHYDRAT_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_c_gc_2
6PGLUCONOLACT_RXN_c_gc_2
FRU_c_gc_Linker_2
2KG_MAL_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
WATER_vc_gc_2
Cl_c_gc_Linker_2
GLUCOKIN_RXN_c_gc_2
SUC_SO4_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
VAL_v_gc_Linker_1
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
K_ec_gc_2
RIBULP3EPIM_RXN_p_gc_2
VAL_v_gc_Linker_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
PGLUCISOM_RXN_p_gc_2
Plastidial_ATP_Synthase_p_gc_2
F16ALDOLASE_RXN_p_gc_2
GLT_xc_gc_2
FRUCTOKINASE_RXN_c_gc_2
2KG_ISOCITRATE_m

In [23]:
# | export

blue_constrained_wt_solution.to_csv(output_dir + "blue_constrained_wt.csv")

#### Starch KO

In [24]:
# | export

with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        blue_constrained_starchko,
        blue_constrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 11:33:32.629413
Running pFBA
Running FVA on 348 reactions using 3 processes
L_ALPHA_ALANINE_v_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
RXN_10773_c_gc_2
L_ALPHA_ALANINE_v_gc_Linker_2
GLC_PROTON_ec_gc_2
MALIC_NADP_RXN_c_gc_2
MAL_SO4_mc_gc_2
MALATE_DEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
DHAP_3PGA_pc_gc_2
GLY_v_gc_Linker_1
H2O_pc_gc_2
GLY_v_gc_Linker_2
GLUC1PURIDYLTRANS_RXN_c_gc_2
H_pc_gc_2
H_mc_gc_2
Cl_c_gc_Linker_1
NADPHox_p_tx_gc_2
2PGADEHYDRAT_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_c_gc_2
6PGLUCONOLACT_RXN_c_gc_2
FRU_c_gc_Linker_2
2KG_MAL_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
WATER_vc_gc_2
Cl_c_gc_Linker_2
GLUCOKIN_RXN_c_gc_2
SUC_SO4_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
VAL_v_gc_Linker_1
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
K_ec_gc_2
RIBULP3EPIM_RXN_p_gc_2
VAL_v_gc_Linker_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
PGLUCISOM_RXN_p_gc_2
Plastidial_ATP_Synthase_p_gc_2
F16ALDOLASE_RXN_p_gc_2
GLT_xc_gc_2
FRUCTOKINASE_RXN_c_gc_2
2KG_ISOCITRATE_m

In [25]:
# | export

blue_constrained_starchko_solution.to_csv(output_dir + "blue_constrained_starchko.csv")

## White Light

### Unconstrained ATPase

#### WT

In [26]:
# | export

with arabidopsis_supermodel.fba_model as m:
    (
        white_unconstrained_wt,
        white_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 11:48:20.644619
Running pFBA
Running FVA on 348 reactions using 3 processes
MAL_ae_gc_2
K_c_gc_Linker_1
METHYLENE_THF_pc_gc_2
MET_v_gc_Linker_2
GLU6PDEHYDROG_RXN_p_gc_2
MALATE_DEH_RXN_c_gc_2
MAL_total_pseudolinker_1
UDPKIN_RXN_c_gc_2
1TRANSKETO_RXN_p_gc_2
K_c_gc_Linker_2
FRU_vc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
SUCROSE_c_gc_Linker_1
MAL_total_pseudolinker_2
ASPAMINOTRANS_RXN_m_gc_2
SER_pc_gc_2
MALIC_NADP_RXN_p_gc_2
MALATE_DEH_RXN_p_gc_2
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_1
SUCROSE_c_gc_Linker_2
PHOSPHORIBULOKINASE_RXN_p_gc_2
OAA_MAL_mc_gc_2
ASPAMINOTRANS_RXN_c_gc_2
RXN_1781_v_gc_2
GLC_ae_gc_2
Cl_ae_gc_2
RXN_2141_p_gc_2
SER_v_gc_Linker_2
GDPKIN_RXN_c_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
LEU_v_gc_Linker_2
RXN490_3650_p_gc_2
CITSYN_RXN_m_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
LYS_v_gc_Linker_1
CIT_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
LYS_v_gc_Linker_2
MAL_c_gc_Linker_2
FRU_total_pseudolinker_1
MAL_c_gc_Linker_1
Cl_ae_gc_1
GLC_ae

In [27]:
# | export

white_unconstrained_wt_solution.to_csv(output_dir + "white_unconstrained_wt.csv")

#### Starch KO

In [28]:
# | export

with arabidopsis_supermodel.fba_model as m:
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        white_unconstrained_starchko,
        white_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 12:05:58.204629
Running pFBA
Running FVA on 348 reactions using 3 processes
L_ALPHA_ALANINE_v_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
RXN_10773_c_gc_2
L_ALPHA_ALANINE_v_gc_Linker_2
GLC_PROTON_ec_gc_2
MALIC_NADP_RXN_c_gc_2
MAL_SO4_mc_gc_2
MALATE_DEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
DHAP_3PGA_pc_gc_2
GLY_v_gc_Linker_1
H2O_pc_gc_2
GLY_v_gc_Linker_2
GLUC1PURIDYLTRANS_RXN_c_gc_2
H_pc_gc_2
H_mc_gc_2
Cl_c_gc_Linker_1
NADPHox_p_tx_gc_2
2PGADEHYDRAT_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_c_gc_2
6PGLUCONOLACT_RXN_c_gc_2
FRU_c_gc_Linker_2
2KG_MAL_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
WATER_vc_gc_2
Cl_c_gc_Linker_2
GLUCOKIN_RXN_c_gc_2
SUC_SO4_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
VAL_v_gc_Linker_1
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
K_ec_gc_2
RIBULP3EPIM_RXN_p_gc_2
VAL_v_gc_Linker_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
PGLUCISOM_RXN_p_gc_2
Plastidial_ATP_Synthase_p_gc_2
F16ALDOLASE_RXN_p_gc_2
GLT_xc_gc_2
FRUCTOKINASE_RXN_c_gc_2
2KG_ISOCITRATE_m

In [29]:
# | export

white_unconstrained_starchko_solution.to_csv(
    output_dir + "white_unconstrained_starchko.csv"
)

### Constrained ATPase

#### WT

In [30]:
# | export

with arabidopsis_supermodel.fba_model as m:
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )

    (
        white_constrained_wt,
        white_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 12:21:43.296282
Running pFBA
Running FVA on 348 reactions using 3 processes
L_ALPHA_ALANINE_v_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
RXN_10773_c_gc_2
L_ALPHA_ALANINE_v_gc_Linker_2
GLC_PROTON_ec_gc_2
MALIC_NADP_RXN_c_gc_2
MAL_SO4_mc_gc_2
MALATE_DEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
DHAP_3PGA_pc_gc_2
GLY_v_gc_Linker_1
H2O_pc_gc_2
GLY_v_gc_Linker_2
GLUC1PURIDYLTRANS_RXN_c_gc_2
H_pc_gc_2
H_mc_gc_2
Cl_c_gc_Linker_1
NADPHox_p_tx_gc_2
2PGADEHYDRAT_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_c_gc_2
6PGLUCONOLACT_RXN_c_gc_2
FRU_c_gc_Linker_2
2KG_MAL_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
WATER_vc_gc_2
Cl_c_gc_Linker_2
GLUCOKIN_RXN_c_gc_2
SUC_SO4_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
VAL_v_gc_Linker_1
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
K_ec_gc_2
RIBULP3EPIM_RXN_p_gc_2
VAL_v_gc_Linker_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
PGLUCISOM_RXN_p_gc_2
Plastidial_ATP_Synthase_p_gc_2
F16ALDOLASE_RXN_p_gc_2
GLT_xc_gc_2
FRUCTOKINASE_RXN_c_gc_2
2KG_ISOCITRATE_m

In [31]:
# | export

white_constrained_wt_solution.to_csv(output_dir + "white_constrained_wt.csv")

#### Starch KO

In [32]:
# | export

with arabidopsis_supermodel.fba_model as m:
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        white_constrained_starchko,
        white_constrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 12:36:07.932328
Running pFBA
Running FVA on 348 reactions using 3 processes
L_ALPHA_ALANINE_v_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
RXN_10773_c_gc_2
L_ALPHA_ALANINE_v_gc_Linker_2
GLC_PROTON_ec_gc_2
MALIC_NADP_RXN_c_gc_2
MAL_SO4_mc_gc_2
MALATE_DEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
DHAP_3PGA_pc_gc_2
GLY_v_gc_Linker_1
H2O_pc_gc_2
GLY_v_gc_Linker_2
GLUC1PURIDYLTRANS_RXN_c_gc_2
H_pc_gc_2
H_mc_gc_2
Cl_c_gc_Linker_1
NADPHox_p_tx_gc_2
2PGADEHYDRAT_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_c_gc_2
6PGLUCONOLACT_RXN_c_gc_2
FRU_c_gc_Linker_2
2KG_MAL_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
WATER_vc_gc_2
Cl_c_gc_Linker_2
GLUCOKIN_RXN_c_gc_2
SUC_SO4_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
VAL_v_gc_Linker_1
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
K_ec_gc_2
RIBULP3EPIM_RXN_p_gc_2
VAL_v_gc_Linker_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
PGLUCISOM_RXN_p_gc_2
Plastidial_ATP_Synthase_p_gc_2
F16ALDOLASE_RXN_p_gc_2
GLT_xc_gc_2
FRUCTOKINASE_RXN_c_gc_2
2KG_ISOCITRATE_m

In [33]:
# | export

white_constrained_starchko_solution.to_csv(
    output_dir + "white_constrained_starchko.csv"
)

## Guard cell has no photosynthetic capacity

### Unconstrained ATPase

#### WT

In [34]:
# | export

with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    (
        nops_unconstrained_wt,
        nops_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 12:50:02.084917
Running pFBA
Running FVA on 348 reactions using 3 processes
MAL_ae_gc_2
K_c_gc_Linker_1
METHYLENE_THF_pc_gc_2
MET_v_gc_Linker_2
GLU6PDEHYDROG_RXN_p_gc_2
MALATE_DEH_RXN_c_gc_2
MAL_total_pseudolinker_1
UDPKIN_RXN_c_gc_2
1TRANSKETO_RXN_p_gc_2
K_c_gc_Linker_2
FRU_vc_gc_2
WATER_ae_gc_3
MAL_ae_gc_3
SUCROSE_c_gc_Linker_1
MAL_total_pseudolinker_2
ASPAMINOTRANS_RXN_m_gc_2
SER_pc_gc_2
MALIC_NADP_RXN_p_gc_2
MALATE_DEH_RXN_p_gc_2
WATER_ae_gc_4
MAL_ae_gc_4
SER_v_gc_Linker_1
SUCROSE_c_gc_Linker_2
PHOSPHORIBULOKINASE_RXN_p_gc_2
OAA_MAL_mc_gc_2
ASPAMINOTRANS_RXN_c_gc_2
RXN_1781_v_gc_2
GLC_ae_gc_2
Cl_ae_gc_2
RXN_2141_p_gc_2
SER_v_gc_Linker_2
GDPKIN_RXN_c_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
LEU_v_gc_Linker_2
RXN490_3650_p_gc_2
CITSYN_RXN_m_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
LYS_v_gc_Linker_1
CIT_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_2
LYS_v_gc_Linker_2
MAL_c_gc_Linker_2
FRU_total_pseudolinker_1
MAL_c_gc_Linker_1
Cl_ae_gc_1
GLC_ae

In [35]:
# | export

nops_unconstrained_wt_solution.to_csv(output_dir + "nops_unconstrained_wt.csv")

#### Starch KO

In [36]:
# | export

with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        nops_unconstrained_starchko,
        nops_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 13:06:44.557252
Running pFBA
Running FVA on 348 reactions using 3 processes
L_ALPHA_ALANINE_v_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
RXN_10773_c_gc_2
L_ALPHA_ALANINE_v_gc_Linker_2
GLC_PROTON_ec_gc_2
MALIC_NADP_RXN_c_gc_2
MAL_SO4_mc_gc_2
MALATE_DEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
DHAP_3PGA_pc_gc_2
GLY_v_gc_Linker_1
H2O_pc_gc_2
GLY_v_gc_Linker_2
GLUC1PURIDYLTRANS_RXN_c_gc_2
H_pc_gc_2
H_mc_gc_2
Cl_c_gc_Linker_1
NADPHox_p_tx_gc_2
2PGADEHYDRAT_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_c_gc_2
6PGLUCONOLACT_RXN_c_gc_2
FRU_c_gc_Linker_2
2KG_MAL_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
WATER_vc_gc_2
Cl_c_gc_Linker_2
GLUCOKIN_RXN_c_gc_2
SUC_SO4_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
VAL_v_gc_Linker_1
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
K_ec_gc_2
RIBULP3EPIM_RXN_p_gc_2
VAL_v_gc_Linker_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
PGLUCISOM_RXN_p_gc_2
Plastidial_ATP_Synthase_p_gc_2
F16ALDOLASE_RXN_p_gc_2
GLT_xc_gc_2
FRUCTOKINASE_RXN_c_gc_2
2KG_ISOCITRATE_m

In [37]:
# | export

nops_unconstrained_starchko_solution.to_csv(
    output_dir + "nops_unconstrained_starchko.csv"
)

### Constrained ATPase

#### WT

- :150 is fine
- 150: is fine

In [23]:
# | export

with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )

    (
        nops_unconstrained_wt_firsthalf,
        nops_unconstrained_wt_solution_firsthalf,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list[:150], processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-08-01 10:19:43.001358
Running pFBA
Running FVA on 150 reactions using 3 processes
GLY_v_gc_Linker_1
PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c_gc_2
K_c_gc_Linker_1
3PGAREARR_RXN_c_gc_2
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
3PGA_Pi_pc_gc_2
GLC_c_gc_Linker_2
FRU_v_gc_Linker_1
VAL_v_gc_Linker_1
F16ALDOLASE_RXN_p_gc_2
CIT_ACONITATE_mc_gc_2
PEPDEPHOS_RXN_p_gc_2
Cl_v_gc_Linker_2
RXN_1827_p_gc_2
PYRUVATE_PROTON_mc_gc_2
Cl_v_gc_Linker_1
UDPKIN_RXN_c_gc_2
OAA_CIT_mc_gc_2
3PGAREARR_RXN_p_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m_gc_2
FRU_PROTON_cv_gc_2
MET_v_gc_Linker_1
MET_v_gc_Linker_2
SER_v_gc_Linker_1
ACONITATEHYDR_RXN_m_gc_2
MAL_total_pseudolinker_1
Photon_tx_gc_2
Cl_c_gc_Linker_2
WATER_ae_gc_3
PGLUCISOM_RXN_c_gc_2
MAL_total_pseudolinker_2
MALIC_NADP_RXN_p_gc_2
ASPAMINOTRANS_RXN_m_gc_2
WATER_ae_gc_4
VAL_v_gc_Linker_2
SUCROSE_c_gc_Linker_1
MAL_ISOCITRATE_mc_gc_2
GLT_xc_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
CO2_mc_gc_2
MAL

In [25]:
# | export

nops_unconstrained_wt_solution_firsthalf.to_csv(
    output_dir + "nops_constrained_wt_first_half.csv"
)

In [18]:
# | export

with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )

    (
        nops_unconstrained_wt_secondhalf,
        nops_unconstrained_wt_solution_secondhalf,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list[150:], processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-08-01 12:36:59.877070
Running pFBA
Running FVA on 198 reactions using 3 processes
OAA_CIT_mc_gc_2
SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m_gc_2
FRU_PROTON_cv_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
MALIC_NAD_RXN_m_gc_2
1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi_gc_2
SEDOBISALDOL_RXN_p_gc_2
GLY_pc_gc_2
Photon_tx_gc_2
MET_v_gc_Linker_2
MAL_total_pseudolinker_1
Cl_c_gc_Linker_2
GLC_PROTON_rev_cv_gc_2
MAL_total_pseudolinker_2
MALIC_NADP_RXN_p_gc_2
ASPAMINOTRANS_RXN_m_gc_2
RXN_10786_c_gc_2
FRUCTOKINASE_RXN_c_gc_2
ISOCITDEH_RXN_c_gc_2
WATER_ae_gc_4
SUCROSE_PHOSPHATASE_RXN_c_gc_2
SUCROSE_SYNTHASE_RXN_c_gc_2
SUCROSE_c_gc_Linker_1
MAL_ISOCITRATE_mc_gc_2
GLT_xc_gc_2
SUCROSE_c_gc_Linker_2
MALATE_DEH_RXN_p_gc_2
PGLUCISOM_RXN_c_gc_2
LYS_v_gc_Linker_1
FRU_total_pseudolinker_1
PROTONATP_rev_vc_gc_2
FRU_total_pseudolinker_2
SUC_ACONITATE_mc_gc_2
LYS_v_gc_Linker_2
2TRANSKETO_RXN_p_gc_2
MAL_c_gc_Linker_1
L_ALPHA_ALANINE_v_gc_Linker_1
6PGLUCONDEHYDROG_RXN_c_gc_2
ACONITATEHYDR_RXN_c_gc

In [18]:
len(set(fva_list[150:]) | set(fva_list[:150]))

348

In [19]:
len(fva_list)

348

In [20]:
len(set(fva_list))

348

In [21]:
# | export

nops_unconstrained_wt_solution_secondhalf.to_csv(
    output_dir + "nops_constrained_wt_second_half.csv"
)

NameError: name 'nops_unconstrained_wt_solution_secondhalf' is not defined

In [22]:
nops_unconstrained_wt_solution_first = pd.read_csv(
    output_dir + "nops_constrained_wt_first_half.csv", index_col=0
)
nops_unconstrained_wt_solution_second = pd.read_csv(
    output_dir + "nops_constrained_wt_second_half.csv", index_col=0
)

In [23]:
test = pd.read_csv(output_dir + "nops_constrained_starchko.csv", index_col=0)

In [24]:
(~test.loc[:, "minimum"].isna()).sum()

348

In [25]:
(~nops_unconstrained_wt_solution_first.loc[:, "minimum"].isna()).sum() + (
    ~nops_unconstrained_wt_solution_second.loc[:, "minimum"].isna()
).sum()

348

Ok, so we have covered the correct number between the two of them

In [26]:
first_index = nops_unconstrained_wt_solution_first[
    ~nops_unconstrained_wt_solution_first.loc[:, "minimum"].isna()
].index

In [27]:
second_index = nops_unconstrained_wt_solution_second[
    ~nops_unconstrained_wt_solution_second.loc[:, "minimum"].isna()
].index

In [28]:
r = fva_list[0]

In [31]:
fva_ids_set = set([r.id for r in fva_list])
len(fva_ids_set)

348

In [37]:
"Photon_tx_gc_2" in [r.id for r in fva_list[:150]]

False

In [38]:
"Photon_tx_gc_2" in [r.id for r in fva_list[150:]]

True

In [45]:
set([r.id for r in fva_list[:150]]) - set(first_index.to_list())

{'2KG_MAL_mc_gc_2',
 '2PGADEHYDRAT_RXN_c_gc_2',
 '2_PERIOD_7_PERIOD_7_PERIOD_34_RXN_c_gc_2',
 'ACONITATEDEHYDR_RXN_c_gc_2',
 'ASPAMINOTRANS_RXN_x_gc_2',
 'ATP_AMP_mc_gc_2',
 'CAII_ae_gc_1',
 'CAII_ae_gc_2',
 'CAII_ae_gc_3',
 'CAII_ae_gc_4',
 'CIT_total_pseudolinker_1',
 'CIT_total_pseudolinker_2',
 'Cl_PROTON_ec_gc_2',
 'EX_X_CO2_t_gc_2',
 'F16ALDOLASE_RXN_c_gc_2',
 'FUMHYDR_RXN_c_gc_2',
 'GAP_Pi_pc_gc_2',
 'GDPKIN_RXN_c_gc_2',
 'GLC_PROTON_ec_gc_2',
 'GLC_total_pseudolinker_1',
 'GLC_total_pseudolinker_2',
 'GLT_v_gc_Linker_1',
 'GLT_v_gc_Linker_2',
 'H2O_mc_gc_2',
 'ILE_v_gc_Linker_1',
 'ILE_v_gc_Linker_2',
 'MALIC_NAD_RXN_m_gc_2',
 'MAL_ACONITATE_mc_gc_2',
 'MAL_SO4_mc_gc_2',
 'MAL_ae_gc_1',
 'MAL_ae_gc_2',
 'MAL_ae_gc_3',
 'MAL_ae_gc_4',
 'MAL_cv_gc_2',
 'MAL_v_gc_Linker_2',
 'NITRATE_ae_gc_1',
 'NITRATE_ae_gc_2',
 'NITRATE_ae_gc_3',
 'NITRATE_ae_gc_4',
 'NITRATE_v_gc_Linker_1',
 'NITRATE_v_gc_Linker_2',
 'OAA_MAL_pc_gc_2',
 'PALMITATE_c_gc_Linker_1',
 'PEPDEPHOS_RXN_c_gc_2',
 'PYR

In [44]:
set(first_index.to_list()) - set([r.id for r in fva_list[:150]])

{'1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p_gc_2',
 '2KG_ACONITATE_mc_gc_2',
 '2KG_CIT_mc_gc_2',
 '2OXOGLUTARATEDEH_RXN_m_gc_2',
 '2_PERIOD_7_PERIOD_1_PERIOD_90_RXN_c_gc_2',
 '2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2',
 '3PGAREARR_RXN_p_gc_2',
 '3PGA_Pi_pc_gc_2',
 '6PGLUCONDEHYDROG_RXN_p_gc_2',
 'ACONITATEHYDR_RXN_m_gc_2',
 'ADENYL_KIN_RXN_m_gc_2',
 'AMP_pc_gc_2',
 'ASPAMINOTRANS_RXN_m_gc_2',
 'CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2',
 'CITSYN_RXN_m_gc_2',
 'CIT_ACONITATE_mc_gc_2',
 'CO2_ec_gc_2',
 'CO2_mc_gc_2',
 'Cl_cv_gc_2',
 'DHAP_Pi_pc_gc_2',
 'F16ALDOLASE_RXN_p_gc_2',
 'FRU_PROTON_cv_gc_2',
 'FRU_PROTON_ec_gc_2',
 'FRU_c_gc_Linker_1',
 'FRU_c_gc_Linker_2',
 'FRU_v_gc_Linker_1',
 'FRU_v_gc_Linker_2',
 'GAPOXNPHOSPHN_RXN_p_gc_2',
 'GLC_PROTON_rev_cv_gc_2',
 'GLC_c_gc_Linker_1',
 'GLC_c_gc_Linker_2',
 'GLC_v_gc_Linker_1',
 'GLC_v_gc_Linker_2',
 'GLU6PDEHYDROG_RXN_p_gc_2',
 'GLYOHMETRANS_RXN_c_gc_2',
 'GLYOHMETRANS_RXN_p_gc_2',
 'GLY_pc_gc_2',
 'H2O_ec_gc_2',
 'H2O_pc_gc_2',
 'K_c_gc_Li

In [36]:
second_index

Index(['2KG_OAA_mc_gc_2', 'Photon_tx_gc_2', 'F16ALDOLASE_RXN_c_gc_2',
       'PYRUVATE_PROTON_mc_gc_2', 'NADH_DEHYDROG_A_RXN_mi_gc_2',
       'RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2', 'FUMHYDR_RXN_c_gc_2',
       'GLT_xc_gc_2', 'ACONITATEHYDR_RXN_c_gc_2', 'PROTONATP_rev_vc_gc_2',
       ...
       'MAL_total_pseudolinker_1', 'MAL_total_pseudolinker_2',
       'FRU_total_pseudolinker_1', 'FRU_total_pseudolinker_2',
       'K_total_pseudolinker_1', 'K_total_pseudolinker_2',
       'Cl_total_pseudolinker_1', 'Cl_total_pseudolinker_2',
       'CIT_total_pseudolinker_1', 'CIT_total_pseudolinker_2'],
      dtype='object', length=198)

In [24]:
nops_unconstrained_wt_solution_first

,fluxes,minimum,maximum
EX_X_pi_t_me_1,0.000000e+00,NaN,NaN
EX_X_pi_t_me_2,0.000000e+00,NaN,NaN
EX_X_pi_t_me_3,0.000000e+00,NaN,NaN
EX_X_pi_t_me_4,0.000000e+00,NaN,NaN
EX_X_pi_t_gc_1,0.000000e+00,NaN,NaN
...,...,...,...
Cl_total_pseudolinker_4,2.266888e-14,NaN,NaN
CIT_total_pseudolinker_1,5.625591e-04,NaN,NaN
CIT_total_pseudolinker_2,0.000000e+00,NaN,NaN
CIT_total_pseudolinker_3,5.625591e-04,NaN,NaN


In [95]:
first_set = set(
    (
        nops_unconstrained_wt_solution_first.loc[:, "minimum"][
            ~nops_unconstrained_wt_solution_first.loc[:, "minimum"].isna()
        ].index
    ).to_list()
)

In [98]:
len(set(fva_list[150:]) - first_set)

198

In [99]:
len(set(fva_list[:150]) - first_set)

150

In [109]:
fva_list_first_set = set([reaction.id for reaction in fva_list[:150]])

In [111]:
len(first_set - fva_list_first_set)

79

In [114]:
len(fva_list_first_set | first_set)

229

In [101]:
set(fva_list[:150])

{<Reaction 1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p_gc_2 at 0x7f75a3d88c40>,
 <Reaction 1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p_gc_2 at 0x7f75a52d1c70>,
 <Reaction 2KG_ACONITATE_mc_gc_2 at 0x7f75a3419ac0>,
 <Reaction 2KG_CIT_mc_gc_2 at 0x7f75a4245af0>,
 <Reaction 2KG_MAL_mc_gc_2 at 0x7f75a397cd90>,
 <Reaction 2KG_SUC_mc_gc_2 at 0x7f75a57b8a90>,
 <Reaction 2OXOGLUTARATEDEH_RXN_m_gc_2 at 0x7f75a4224a00>,
 <Reaction 2_PERIOD_4_PERIOD_1_PERIOD_111_RXN_c_gc_2 at 0x7f75a4ee16d0>,
 <Reaction 2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2 at 0x7f75a5041af0>,
 <Reaction 3PGAREARR_RXN_p_gc_2 at 0x7f75a5401f70>,
 <Reaction 3PGA_Pi_pc_gc_2 at 0x7f75a50e5e20>,
 <Reaction 4_AMINO_BUTYRATE_v_gc_Linker_1 at 0x7f75a2ff56a0>,
 <Reaction 4_AMINO_BUTYRATE_v_gc_Linker_2 at 0x7f75a2ff57c0>,
 <Reaction 6PGLUCONDEHYDROG_RXN_p_gc_2 at 0x7f75a3a205b0>,
 <Reaction 6PGLUCONOLACT_RXN_p_gc_2 at 0x7f75a4be8430>,
 <Reaction ACONITATEDEHYDR_RXN_c_gc_2 at 0x7f75a4fa8f10>,
 <Reaction ACONITATEDEHYDR_RXN_m_gc_2 at 0x7f75a5054a00>,
 <Re

In [74]:
(~nops_unconstrained_wt_solution_first.loc[:, "minimum"].isna()).sum()

150

In [75]:
nops_unconstrained_wt_solution_first[
    ~nops_unconstrained_wt_solution_first.loc[:, "minimum"].isna()
]

,fluxes,minimum,maximum
F16ALDOLASE_RXN_p_gc_2,0.000000e+00,0.000000,0.440304
Photon_tx_gc_2,0.000000e+00,0.000000,0.000000
PYRUVATE_PROTON_mc_gc_2,0.000000e+00,-0.993966,0.000000
NADH_DEHYDROG_A_RXN_mi_gc_2,2.250237e-03,0.000000,4.746712
RXN_1841_v_gc_2,0.000000e+00,0.000000,0.000000
...,...,...,...
FRU_total_pseudolinker_2,3.768946e-03,0.000000,0.054922
K_total_pseudolinker_1,3.578323e-03,0.000000,0.030914
K_total_pseudolinker_2,7.916723e-03,0.000000,0.035252
Cl_total_pseudolinker_1,2.266888e-14,0.000000,0.019680


In [77]:
nops_unconstrained_wt_solution_second[
    ~nops_unconstrained_wt_solution_second.loc[:, "minimum"].isna()
]

,fluxes,minimum,maximum
2KG_OAA_mc_gc_2,0.000000e+00,0.000000e+00,1000.000000
Photon_tx_gc_2,0.000000e+00,0.000000e+00,0.000000
F16ALDOLASE_RXN_c_gc_2,2.812796e-04,0.000000e+00,0.440961
PYRUVATE_PROTON_mc_gc_2,0.000000e+00,-9.939659e-01,0.000000
NADH_DEHYDROG_A_RXN_mi_gc_2,2.250237e-03,0.000000e+00,4.746712
...,...,...,...
K_total_pseudolinker_2,7.916723e-03,0.000000e+00,0.035252
Cl_total_pseudolinker_1,2.266888e-14,0.000000e+00,0.019680
Cl_total_pseudolinker_2,4.338400e-03,0.000000e+00,0.024018
CIT_total_pseudolinker_1,5.625591e-04,0.000000e+00,0.010896


In [68]:
nops_unconstrained_wt_solution_second.loc[:, "minimum"].isna()

EX_X_pi_t_me_1               True
EX_X_pi_t_me_2               True
EX_X_pi_t_me_3               True
EX_X_pi_t_me_4               True
EX_X_pi_t_gc_1               True
                            ...  
Cl_total_pseudolinker_4      True
CIT_total_pseudolinker_1    False
CIT_total_pseudolinker_2    False
CIT_total_pseudolinker_3     True
CIT_total_pseudolinker_4     True
Name: minimum, Length: 7097, dtype: bool

In [66]:
(
    (
        ~nops_unconstrained_wt_solution_first.loc[:, "minimum"].isna()
        == ~nops_unconstrained_wt_solution_second.loc[:, "minimum"].isna()
    )
).sum()

6907

In [78]:
nops_unconstrained_wt_solution = nops_unconstrained_wt_solution_first.combine_first(
    nops_unconstrained_wt_solution_second
)

In [79]:
(~nops_unconstrained_wt_solution.loc[:, "minimum"].isna()).sum()

269

In [59]:
help(pd.DataFrame.combine_first)

Help on function combine_first in module pandas.core.frame:

combine_first(self, other: 'DataFrame') -> 'DataFrame'
    Update null elements with value in the same location in `other`.
    
    Combine two DataFrame objects by filling null values in one DataFrame
    with non-null values from other DataFrame. The row and column indexes
    of the resulting DataFrame will be the union of the two. The resulting
    dataframe contains the 'first' dataframe values and overrides the
    second one values where both first.loc[index, col] and
    second.loc[index, col] are not missing values, upon calling
    first.combine_first(second).
    
    Parameters
    ----------
    other : DataFrame
        Provided DataFrame to use to fill null values.
    
    Returns
    -------
    DataFrame
        The result of combining the provided DataFrame with the other object.
    
    See Also
    --------
    DataFrame.combine : Perform series-wise operation on two DataFrames
        using a given fun

#### Starch KO

In [39]:
# | export

with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        nops_unconstrained_starchko,
        nops_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes
    )

Started running pFBA (and FVA) @ 2023-07-31 15:57:41.997429
Running pFBA
Running FVA on 348 reactions using 3 processes
L_ALPHA_ALANINE_v_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
RXN_10773_c_gc_2
L_ALPHA_ALANINE_v_gc_Linker_2
GLC_PROTON_ec_gc_2
MALIC_NADP_RXN_c_gc_2
MAL_SO4_mc_gc_2
MALATE_DEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
DHAP_3PGA_pc_gc_2
GLY_v_gc_Linker_1
H2O_pc_gc_2
GLY_v_gc_Linker_2
GLUC1PURIDYLTRANS_RXN_c_gc_2
H_pc_gc_2
H_mc_gc_2
Cl_c_gc_Linker_1
NADPHox_p_tx_gc_2
2PGADEHYDRAT_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_c_gc_2
6PGLUCONOLACT_RXN_c_gc_2
FRU_c_gc_Linker_2
2KG_MAL_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
WATER_vc_gc_2
Cl_c_gc_Linker_2
GLUCOKIN_RXN_c_gc_2
SUC_SO4_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
VAL_v_gc_Linker_1
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
K_ec_gc_2
RIBULP3EPIM_RXN_p_gc_2
VAL_v_gc_Linker_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
PGLUCISOM_RXN_p_gc_2
Plastidial_ATP_Synthase_p_gc_2
F16ALDOLASE_RXN_p_gc_2
GLT_xc_gc_2
FRUCTOKINASE_RXN_c_gc_2
2KG_ISOCITRATE_m

In [40]:
# | export

nops_unconstrained_starchko_solution.to_csv(
    output_dir + "nops_constrained_starchko.csv"
)